Merhaba,

Bu çalışmada COVID-19 hastalığının, öldürücülük ve iyileşme durumlarına; yaş ve cinsiyet faktörünün etkileri incelenmiştir. Çalışmada İtalya, Almanya, İspanya ülkeleri ele alınmıştır.

* İtalya için; "Novel Coronavirus COVID-19 Italy Dataset" datasetinin "covid-age.csv" dosyasından,
* Almanya için; "COVID-19 Tracking Germany" datasetinin "covid_de.csv" dosyasından,
* İspanya için; "COVID-19 in Spain" datasetinin "nacional_covid19_rango_edad.csv" dosyasından istifade edilmiştir.


İncelemenin ana hatları aşağıda listelenmiştir. Her maddede işlem öncesinde datanın kullanışlı hale getirilmesi için öncalışma yapılmıştır. İncelemeler python dili kullanılarak yapılmıştır.

a- Total ölümler üzerinden yaş etkisinin grafikler üzerinden gösterimi,

b- Ölümlerde cinsiyet etkisinin grafikler üzerinden gösterimi.

# **a-Kullanılacak kütüphanelerin tanımlanması, datasetlerinin tanımlanması**

# **a.1-Kütüphanelerin tanımlanması ve dosya yollarının oluşturulması**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns  # visualization tool

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

import os
for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

# **a.2-Data setlerin tanımlanıp parametrelere aktarılması**

Bu kısımda başta belirttiğim data setlerinin ilgili dosyalarını her ülke için ayrı parametre oluşturarak tanımladım.

In [ ]:
dataitaly = pd.read_csv('/kaggle/input/italy-covid19/covid-age.csv')
datagermany = pd.read_csv('/kaggle/input/covid19-tracking-germany/covid_de.csv')
dataspain = pd.read_csv('/kaggle/input/covid19-in-spain/nacional_covid19_rango_edad.csv')

In [ ]:
dataitaly.age_classes.unique() #komutu ile datasetimizin yaş grubu kolonundaki data tiplerini listeliyoruz
datagermany.gender.unique()    #unique her data örneğinden tek birtane göstermeye yarar
dataspain.rango_edad.unique()

In [ ]:
#NaN yani number olmayan anlamında belirsiz gelen satırları bu dropna komutuyla temizledik.
dataitaly = dataitaly.dropna()
datagermany = datagermany.dropna()
dataspain = dataspain.dropna()

In [ ]:
#total satırı listedeki her günün yaş gruplarına göre ölümlerinin toplamını gösteriyor. Bu bilgi incelememizde bize
#lazım olmadığı gibi grafikte hhatalı gösterme neden olacaktır.
#80 y + yazan satırlar net olmayan bilgiler için girilmiş ancak gösterimde sorun yaratacağı için gözardı ediyoruz.
for i in range(len(dataspain.index)):
    if (dataspain.at[i,"rango_edad"] == "Total" or dataspain.at[i,"rango_edad"] == "80 y +"):
        dataspain = dataspain.drop(i)

dataspain.reset_index(inplace=True)

In [ ]:
#yaşı bilinmeyen örnekleri göz ardı edeceğiz
for i in range(len(dataitaly.index)):
    if (dataitaly.at[i,"age_classes"] == "Uknown"):
        dataitaly = dataitaly.drop(i)

dataitaly.reset_index(inplace=True)

In [ ]:
#sort_values komutu ile yaş grubu sıralamasını düzeltiyoruz. aksi halde karışık görünecekti.

dataitaly = dataitaly.sort_values(by=["age_classes"])
datagermany = datagermany.sort_values(by=["age_group"])
dataspain = dataspain.sort_values(by=["rango_edad"])

In [ ]:
#boş olan yada istemediğimiz diğer satırları sildiğimiz ve sıralamaları da değiştirdiğimiz
#için datasetlerin indekslerini topluca güncelliyoruz.

dataitaly.reset_index(inplace=True)
datagermany.reset_index(inplace=True)
dataspain.reset_index(inplace=True)

# **a.3.1-İtalya datasetinin oluşturulması**

**a.3-Verilerin kullanışlı duruma getirilmesi, datasetlerden kendi datasetimizi oluşturma ve görselleştirme**

Öncelikle data setlerimizi oluşturup ardından grafik ortamına akataracağız

In [ ]:
dataitaly.head(20)

In [ ]:
ditaly = dataitaly[["age_classes","total_deaths"]]#sadece benim istediğim 2 sütundan oluşan yeni bir data seti oluşturdum.
ditaly

# **a.3.2-Almanya datasetinin oluşturulması**

Almanya dosyası italyada olduğu gibi yaş gruplarına göre toplanarak düzenlenmiş halde değil. Gün gün üstelik yaşlara göre bölünmüş halde olan listeyi yaş gruplarını içeren "age_group" sütununa göre gruplandıracağız. İtalyada olduğu gibi şimdilik cinsiyeti esaba katmayacağız.

In [ ]:
#ilk olarak yaş ve ölüm sütunlarını alıyoruz
dgermany = datagermany[["age_group","deaths"]]

#ardından datasetimizi gruplanmış hale çevirerek başka bir data sete aktarıyoruz
dgerwithgroup = dgermany.groupby("age_group") 

#gruplanmış data setini toplayarak çağırınca italya örneğinde olduğu gibi düzenli bir dataset elde etmiş olacağız.
#bu halini baştaki datasetimize tekrar yazıyoruz
dgermany = dgerwithgroup.sum()
dgermany

# **a.3.3-İspanya datasetinin oluşturulması**
rango_edad=yaş grubu,
sexo=cinsiyet,
casos_confirmados=onaylanmış vakalar,
fallecidos=ölüm.


İspanya dosyasında da yine Almanya örneğinde olduğu gibi datalar gün olarak ayrılmış haldedir. Bu yüzden aynı yöntemle düzenleyeceğiz.

In [ ]:
dspain = dataspain[["rango_edad","fallecidos"]]
dspawithgroup = dspain.groupby("rango_edad") 
dspain  = dspawithgroup.sum()
dspain

# a.4- Total ölümler üzerinden yaş etkisinin grafikler üzerinden gösterimi

Önceden oluşturduğumuz İtalya data seti aşağıda nokta grafiği(scatter) ile görselleştirilmiştir.

In [ ]:
fig, axs = plt.subplots(figsize=(8,8))

axs.scatter(ditaly["age_classes"], ditaly["total_deaths"],color = 'red')
axs.set(xlabel="Yaş Grupları",
       ylabel="Ölüm Sayıları",
       title="Toplam ölümlerin yaş gruplarına göre dağılımı(italya)")
plt.show()

Aynı örnekten fazlaca verinin olmadığı datasetlerde scatter grafiği pek açıklayıcı görünmemektedir. Aynı örnek bar grafik ile aşağıda yapılmıştır.

In [ ]:
plt.bar(ditaly["age_classes"], ditaly["total_deaths"],width = 0.8, color = ['blue']) 
  
plt.xlabel('yaş grupları') 
plt.ylabel('ölüm sayıları') 
plt.title('İtalya ölüm oranları bar grafik gösterimi') 

plt.show() 

Aynı grafiği diğer data setleriyle de yapmak mümkündür. Aşağıda her üçünün yanyana tek seferde gösterimi yapılmıştır.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 5))
ditaly.plot(kind = 'line',x="age_classes", y="total_deaths", ax=ax1)
dgermany.plot(ax=ax2)
dspain.plot(ax=ax3)
plt.show()

Yukarıdaki grafiklerde yaş ilerledikçe ölüm sayılarının arttığını görüyoruz. 90+ larda ölüm sayılarının düşmesi veri 90 üstü çok fazla kişinin zaten olmamasından dolayı ölüm sayılarının da düşük olmasıdır.

# b- Ölümlerde cinsiyet etkisinin grafikler üzerinden gösterimi

# b.1-Datasetlerin oluşturulması


#  b.1.1-İtalya cinsiyete göre dataseti

İtalya datasetindeki veriler yine istediğiiz gibi gruplu olduğu için sadece istediğimiz sütunları içeren 1 erkek 1 kadın için toplamda 2 tane yeni dataseti oluşturduk.

In [ ]:
ditalywoman = dataitaly[["age_classes","female_deaths"]]
ditalyman = dataitaly[["age_classes","male_deaths"]]


# b.1.2-Almanya cinsiyete göre dataseti
Bu aşama önceden kullanılan kopyalama metotları yerine döngülerle satır ekleyerek yapılacaktır.
Aşağıda Almanya datasetinin sütunları görülmektedir. En soldaki ismi yazmayan index sütununu saymadan 0 dan başlayarak sayarsak yaş grubu(age_group) sütunu 3, cinsiyet(gender) sütunu 4, ölümler(deaths) sütunu 7 inci sütundur.

In [ ]:
#dgermany = datagermany[["age_group","gender","deaths"]] 

#şeklinde belli sütunları direk dataframeimize kopyalamak yerine

In [ ]:
#dgermany = datagermany.iloc[[5],[2,3,6]] # şeklinde ifade kullanacağız
dgermany #bu şekilde 5 inci indisteki satırın 2, 3, ve 6'ıncı satırını almış olduk. 

Şimdide satır numarasını tek tek arttırarark bütün satırları sırayla dataframeimizin içine aktararak yeni datasetimizi oluşturacağız. Bu defa 1 tane kadınlar 1 tane de erkekler için iki ayrı dataseti oluşturacağız.

In [ ]:
#dgermanywoman = dgermanywoman.drop(dgermanywoman.index, inplace=True) #eğer doluysa önce dgermanywoman data setimizi tamamen boşaltıyoruz.
#***yukarıdaki kod ilk çalıştırmada hata verecektir. sebebi boşaltmak için böyle bir dataframe in zaten hiç olmamasıdır.***
dgermanywoman = pd.DataFrame(columns=["age_group","gender","deaths"]) #ardından data setimizin sütunlarını belirliyoruz.

#aynı işlemleri birde erkeklerin dataseti için yapacağız.
#dgermanyman = dgermanyman.drop(dgermanyman.index, inplace=True)
#***yukarıdaki kod ilk çalıştırmada hata verecektir. sebebi boşaltmak için böyle bir dataframe in zaten hiç olmamasıdır.***
dgermanyman = pd.DataFrame(columns=["age_group","gender","deaths"])

In [ ]:
for i in range(len(datagermany.index)):
#döngü uzunluğunu ana datasetimizin satır uzunluğu kadar dönecek şekilde ayarlandı
    
    if (datagermany.at[i,"gender"] == "F" and datagermany.at[i,"deaths"] > 0):
    #at komutu ile index belirterek ismini
    #verdiğimiz sütunun değerini(value) alırız.Cinsiyeti F olan ölüm sayısı 0 dan büyük olanları hesaba katıyoruz
    
            bul=0
            #bulma değişkenimmizi döngü başında 0 yapıyoruz. Aşağıda kullanacağız
        
            for j in range(len(dgermanywoman.index)):
                #bu sefer yeni dataframe imimizin içinde dönüyoruz
                
                if datagermany.at[i,"age_group"] == dgermanywoman.at[j,"age_group"]:
                    #ana datasetimizde sıralı giderken, elimizdeki yaşgrubunu yeni data setinde gezzerek arıyoruz.
                    
                    bul = 1
                    #Eğer bulduysay bul u 1 yapıyoruz. Bulamadıysak zaten 0 dı.
                    
                    dgermanywoman.at[j,"deaths"] = dgermanywoman.at[j,"deaths"] + datagermany.at[i,"deaths"]
                    #yeni datasette bulduğumuz satırdaki değer ile ana datasetten gelen değeri toplayıp yeni datasette
                    #olduğumuz satıra yazacağız
                    
                    break
                    #sonuç bulduktan sonra aramya devam etmemesini sağlar
            
            if bul == 0:
            #eğer bul 1 olmamamışsa yani isteğimiz yaş grubu satırı yoksa ana datasetten gelen satırı direk ekliyoruz.
                    dgermanywoman = dgermanywoman.append(datagermany.iloc[[i],[3,4,7]],ignore_index=True)
                    #F olan değerler female yani
        
        #kadınları belirtmektedir. ignore_index in true olması eklediği satırlara yeniden sıralı numara vermesi içindir.
        #false yaparsak ki virgülden sonrasını hiç yazmasak defaultda falsedır. Geldikleri yerdeki indeksleriyle görünürler.
    
    
    elif (datagermany.at[i,"gender"] == "M" and datagermany.at[i,"deaths"] != 0):
            #M tipi male yani ekrek demektir.
            
            bul=0
            for j in range(len(dgermanyman.index)):
                if datagermany.at[i,"age_group"] == dgermanyman.at[j,"age_group"]:
                    bul = 1
                    dgermanyman.at[j,"deaths"] = dgermanyman.at[j,"deaths"] + datagermany.at[i,"deaths"]
                    break
            if bul == 0:
                dgermanyman = dgermanyman.append(datagermany.iloc[[i],[3,4,7]],ignore_index=True)
                

Yukarıdaki işlem oldukça performanssızdır ve orjinal datasetinin büyük olmasından dolayı sonuçlanması zaman alır ancak doğrudan kopyalama methotlarıyla aynı sonucu üretir.

# b.1.3 İspanya cinsiyete göre dataseti

sexo: cinsiyet,
hombres:erkekler,
mujeres:kadınlar,
ambos: her ikisi.

In [ ]:

#dspainwoman = dspainwoman.drop(dspainwoman.index, inplace=True)
#***bu kod ilk çalıştırmada hata verecektir. Sebebi boşaltmak için böyle bir dataframe in zaten hiç olmamasıdır.***
dspainwoman = pd.DataFrame(columns=["rango_edad","sexo","fallecidos"]) 

#dspainman = dspainman.drop(dspainman.index, inplace=True)
#***bu kod ilk çalıştırmada hata verecektir. Sebebi boşaltmak için böyle bir dataframe in zaten hiç olmamasıdır.***
dspainman = pd.DataFrame(columns=["rango_edad","sexo","fallecidos"])

#Bu örnektecinsiyet ayrımı yaptığımız için ambos seçeneğini liste dışı bırakacağız.
#Yaşı gösteren rango_edad sütunu 1'inci, cinsiyet sütunu sexo 2'inci, ölümler(fallecidos) sütunu 6 inci indistedir.
for i in range(len(dataspain.index)):
    if (dataspain.at[i,"sexo"] == "mujeres" and dataspain.at[i,"fallecidos"] > 0):
            bul=0
            for j in range(len(dspainwoman.index)):
                if dataspain.at[i,"rango_edad"] == dspainwoman.at[j,"rango_edad"]:
                    bul = 1
                    dspainwoman.at[j,"fallecidos"] = dspainwoman.at[j,"fallecidos"] + dataspain.at[i,"fallecidos"]
                    break
            if bul == 0:
                    dspainwoman = dspainwoman.append(dataspain.iloc[[i],[3,4,8]],ignore_index=True)    
    elif (dataspain.at[i,"sexo"] == "hombres" and dataspain.at[i,"fallecidos"] > 0):
            bul=0
            for j in range(len(dspainman.index)):
                if dataspain.at[i,"rango_edad"] == dspainman.at[j,"rango_edad"]:
                    bul = 1
                    dspainman.at[j,"fallecidos"] = dspainman.at[j,"fallecidos"] + dataspain.at[i,"fallecidos"]
                    break
            if bul == 0:
                dspainman = dspainman.append(dataspain.iloc[[i],[3,4,8]],ignore_index=True)

In [ ]:
#Örnek olarak erkekleri içeren dataseti dspainman'in sexosütununda hombres olmayan erkek olmayan sonuçları
#listelemek istersek 0 sonuç alırız != ifadesini == e çevirirsek bütün satırlar gelir.

dspainman[dspainman["sexo"] != "hombres"] 

In [ ]:
#Son olarak bütün datasetlerimizi yaş grubu sütunlarına göre yeniden düzenliyoruz.
#Aksi halde eklenme sırasına göre görünecekler.

ditalywoman = ditalywoman.sort_values(by=["age_classes"])
ditalyman = ditalyman.sort_values(by=["age_classes"])
dgermanywoman = dgermanywoman.sort_values(by=["age_group"])
dgermanyman = dgermanyman.sort_values(by=["age_group"])
dspainwoman = dspainwoman.sort_values(by=["rango_edad"])
dspainman = dspainman.sort_values(by=["rango_edad"])

# b.2-Cinsiyet ayrımlı datasetlerin grafik üzerinde gösterimi

#  b.2.1-İtalya karşılaştırma grafiği

In [ ]:
fig, (axs1,axs2) = plt.subplots(1, 2, figsize=(20, 10))
ditalywoman.plot(kind = "bar",color="red",x="age_classes", y="female_deaths", ax=axs1)
ditalyman.plot(kind = "bar",x="age_classes", y="male_deaths", ax=axs2)
plt.show()

30 yaştan itibaren erkek ölümleri her yaş gurubunda daha yüksek görülmektedir.

# b.2.2-Almanyanın karşılaştırma grafiği

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
dgermanywoman.plot(kind = 'bar',x="age_group", y="deaths", ax=ax1,color = "red")
dgermanyman.plot(kind = 'bar',x="age_group", y="deaths", ax=ax2,color = "blue")
plt.show()

Bar grafikle daha da iyi görünüyor ki  80 yaş ve üstünde ölümler yakın iken diğer 60-80 yaş arası erkekler daha büyük bir risk altındadır.

# b.2.3- Meksika karşılaştırma grafiği

In [ ]:
dger = pd.concat([dspainwoman, dspainman], axis=0, ignore_index=False)
sns.factorplot(x='rango_edad', y='fallecidos', hue='sexo', kind='bar', data=dger)
plt.show()

In [ ]:
Ortak grafikte gösterimi aşağıdaki gibidir.

In [ ]:
fig, (axs1, axs2) = plt.subplots(1, 2, figsize=(20, 10))
axs1.bar(dspainwoman["rango_edad"], dspainwoman["fallecidos"],color = "red")
axs2.bar(dspainman["rango_edad"], dspainman["fallecidos"],color = "blue")
axs1.set(xlabel="Yaş Grupları", ylabel="Ölüm Sayıları", title="Kadın ölümlerinin yaş gruplarına göre dağılımı(ispanya)")
axs2.set(xlabel="Yaş Grupları", ylabel="Ölüm Sayıları", title="Erkek ölümlerin yaş gruplarına göre dağılımı(ispanya)")
plt.show()

Meksikada da erkek ölümleri açık ara farkla önde.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 5))
ditalywoman.plot(kind = 'line',x="age_classes", y="female_deaths", ax=ax1)
ditalyman.plot(kind = 'line',x="age_classes", y="male_deaths", ax=ax1)

dgermanywoman.plot(kind = 'line',x="age_group", y="deaths", ax=ax2)
dgermanyman.plot(kind = 'line',x="age_group", y="deaths", ax=ax2)

dspainwoman.plot(kind = 'line',x="rango_edad", y="fallecidos", ax=ax3)
dspainman.plot(kind = 'line',x="rango_edad", y="fallecidos", ax=ax3)

plt.show()

3 ülkeninde karşılaştırmasına bakarak virüsün erkek ve yaşlılarda daha etkili olduğunu ancak 90 yaş üstünde cinsyetin artık belirleyici olmaktan çıktığını söyleyebiliriz.